In [1]:
!pip install --upgrade google-cloud-aiplatform

In [2]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel

In [3]:
from google.oauth2 import service_account

In [6]:
credentials = service_account.Credentials.from_service_account_file('/content/healthy-garage-245205-354224964de0.json')

In [7]:
credentials

In [ ]:
vertexai.init(project="healthy-garage-245205", location="us-central1", credentials=credentials)

In [ ]:
model = GenerativeModel("gemini-1.0-pro-001")

In [ ]:
response = model.generate_content(
    "What's the exchange rate for euros to dollars today?"
)
print(response.text)

I do not have access to real-time information and do not have the capability to provide you with the current exchange rate. 

For the most up-to-date and accurate information, I recommend checking a reliable financial website or using a currency converter.


In [ ]:
import requests
url = "https://api.frankfurter.app/latest"
response = requests.get(url)
data = response.json()
data

{'amount': 1.0,
 'base': 'EUR',
 'date': '2024-03-15',
 'rates': {'AUD': 1.6579,
  'BGN': 1.9558,
  'BRL': 5.4461,
  'CAD': 1.4731,
  'CHF': 0.9613,
  'CNY': 7.838,
  'CZK': 25.166,
  'DKK': 7.4571,
  'GBP': 0.8541,
  'HKD': 8.5199,
  'HUF': 393.2,
  'IDR': 17011,
  'ILS': 3.9811,
  'INR': 90.26,
  'ISK': 148.9,
  'JPY': 162.03,
  'KRW': 1448.71,
  'MXN': 18.1915,
  'MYR': 5.1241,
  'NOK': 11.5205,
  'NZD': 1.786,
  'PHP': 60.494,
  'PLN': 4.2953,
  'RON': 4.9711,
  'SEK': 11.2674,
  'SGD': 1.4562,
  'THB': 39.053,
  'TRY': 35.092,
  'USD': 1.0892,
  'ZAR': 20.352}}

In [ ]:
from vertexai.preview.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
)

In [ ]:
get_exchange_rate_func = FunctionDeclaration(
    name="get_exchange_rate",
    description="Get the exchange rate for currencies between countries",
    parameters={
    "type": "object",
    "properties": {
        "currency_date": {
            "type": "string",
            "description": "A date that must always be in YYYY-MM-DD format or the value 'latest' if a time period is not specified"
        },
        "currency_from": {
            "type": "string",
            "description": "The currency to convert from in ISO 4217 format"
        },
        "currency_to": {
            "type": "string",
            "description": "The currency to convert to in ISO 4217 format"
        }
    },
         "required": [
            "currency_from",
            "currency_date",
      ]
  },
)

In [ ]:
exchange_rate_tool = Tool(
    function_declarations=[get_exchange_rate_func],
)

In [ ]:
prompt = """What is the exchange rate from Australian dollars to Swedish krona?
How much is 500 Australian dollars worth in Swedish krona?"""

response = model.generate_content(
    prompt,
    tools=[exchange_rate_tool],
)

In [ ]:
print(response.candidates[0].content)

role: "model"
parts {
  function_call {
    name: "get_exchange_rate"
    args {
      fields {
        key: "currency_date"
        value {
          string_value: "latest"
        }
      }
      fields {
        key: "currency_from"
        value {
          string_value: "AUD"
        }
      }
      fields {
        key: "currency_to"
        value {
          string_value: "SEK"
        }
      }
    }
  }
}



In [ ]:
params = {}
for key, value in response.candidates[0].content.parts[0].function_call.args.items():
    params[key[9:]] = value
params

{'date': 'latest', 'to': 'SEK', 'from': 'AUD'}

In [ ]:
import requests
url = f"https://api.frankfurter.app/{params['date']}"
api_response = requests.get(url, params=params)
api_response.json()

{'amount': 1.0, 'base': 'AUD', 'date': '2024-03-15', 'rates': {'SEK': 6.7962}}

In [ ]:
response = model.generate_content(
    [
    Content(role="user", parts=[
        Part.from_text(prompt + """Give your answer in steps with lots of detail
            and context, including the exchange rate and date."""),
    ]),
    Content(role="function", parts=[
        Part.from_dict({
            "function_call": {
                "name": "get_exchange_rate",
            }
        })
    ]),
    Content(role="function", parts=[
        Part.from_function_response(
            name="get_exchange_rate",
            response={
                "content": api_response.text,
            }
        )
    ]),
    ],
    tools=[exchange_rate_tool],
)

In [ ]:
response.candidates[0].content.parts[0].text

'The exchange rate from Australian dollars to Swedish krona on 2024-03-15 is 6.7962.\nTherefore, 500 Australian dollars is worth 500 * 6.7962 = 3398.1 Swedish krona.'